0.ipynb：一篇新闻文章，有一个（问题，答案）对 - 简单RAG - Ragas评估

# 加载环境变量

In [1]:
import os
from dotenv import find_dotenv, load_dotenv

# 加载环境变量
load_dotenv(find_dotenv())

True

# 源数据

data from [CRUD-RAG：大型语言模型检索增强生成的综合中文基准](https://github.com/IAAR-Shanghai/CRUD_RAG/tree/main)

In [2]:
origin_data = {
    "ID": "64fa9b27b82641eb8ecbe14c",
    "event": "2023年7月28日，国家卫生健康委在全国范围内开展“启明行动”——防控儿童青少年近视健康促进活动，发布《防控儿童青少年近视核心知识十条》。",
    "news1": "2023-07-28 10:14:27作者：白剑峰来源：人民日报    ，正文：为在全社会形成重视儿童眼健康的良好氛围，持续推进综合防控儿童青少年近视工作落实，国家卫生健康委决定在全国持续开展“启明行动”——防控儿童青少年近视健康促进活动，并发布了《防控儿童青少年近视核心知识十条》。本次活动的主题为：重视儿童眼保健，守护孩子明眸“视”界。强调预防为主，推动关口前移，倡导和推动家庭及全社会共同行动起来，营造爱眼护眼的视觉友好环境，共同呵护好孩子的眼睛，让他们拥有一个光明的未来。国家卫生健康委要求，开展社会宣传和健康教育。充分利用网络、广播电视、报纸杂志、海报墙报、培训讲座等多种形式，广泛开展宣传倡导，向社会公众传播开展儿童眼保健、保护儿童视力健康的重要意义，以《防控儿童青少年近视核心知识十条》为重点普及预防近视科学知识。创新健康教育方式和载体，开发制作群众喜闻乐见的健康教育科普作品，利用互联网媒体扩大传播效果，提高健康教育的针对性、精准性和实效性。指导相关医疗机构将儿童眼保健和近视防控等科学知识纳入孕妇学校、家长课堂内容。开展儿童眼保健及视力检查咨询指导。医疗机构要以儿童家长和养育人为重点，结合眼保健和眼科临床服务，开展个性化咨询指导。要针对儿童常见眼病和近视防控等重点问题，通过面对面咨询指导，引导儿童家长树立近视防控意识，改变不良生活方式，加强户外活动，养成爱眼护眼健康行为习惯。提高儿童眼保健专科服务能力。各地要积极推进儿童眼保健专科建设，扎实组织好妇幼健康职业技能竞赛“儿童眼保健”项目，推动各层级开展比武练兵，提升业务能力。",
    "questions": "国家卫生健康委在2023年7月28日开展的“启明行动”是为了防控哪个群体的哪种健康问题，并请列出活动发布的指导性文件名称。",
    "answers": "“启明行动”是为了防控儿童青少年的近视问题，并发布了《防控儿童青少年近视核心知识十条》。"
}

In [3]:
print(origin_data['news1'], "\n")
print(origin_data['questions'], "\n")
print(origin_data['answers'], "\n")

2023-07-28 10:14:27作者：白剑峰来源：人民日报    ，正文：为在全社会形成重视儿童眼健康的良好氛围，持续推进综合防控儿童青少年近视工作落实，国家卫生健康委决定在全国持续开展“启明行动”——防控儿童青少年近视健康促进活动，并发布了《防控儿童青少年近视核心知识十条》。本次活动的主题为：重视儿童眼保健，守护孩子明眸“视”界。强调预防为主，推动关口前移，倡导和推动家庭及全社会共同行动起来，营造爱眼护眼的视觉友好环境，共同呵护好孩子的眼睛，让他们拥有一个光明的未来。国家卫生健康委要求，开展社会宣传和健康教育。充分利用网络、广播电视、报纸杂志、海报墙报、培训讲座等多种形式，广泛开展宣传倡导，向社会公众传播开展儿童眼保健、保护儿童视力健康的重要意义，以《防控儿童青少年近视核心知识十条》为重点普及预防近视科学知识。创新健康教育方式和载体，开发制作群众喜闻乐见的健康教育科普作品，利用互联网媒体扩大传播效果，提高健康教育的针对性、精准性和实效性。指导相关医疗机构将儿童眼保健和近视防控等科学知识纳入孕妇学校、家长课堂内容。开展儿童眼保健及视力检查咨询指导。医疗机构要以儿童家长和养育人为重点，结合眼保健和眼科临床服务，开展个性化咨询指导。要针对儿童常见眼病和近视防控等重点问题，通过面对面咨询指导，引导儿童家长树立近视防控意识，改变不良生活方式，加强户外活动，养成爱眼护眼健康行为习惯。提高儿童眼保健专科服务能力。各地要积极推进儿童眼保健专科建设，扎实组织好妇幼健康职业技能竞赛“儿童眼保健”项目，推动各层级开展比武练兵，提升业务能力。 

国家卫生健康委在2023年7月28日开展的“启明行动”是为了防控哪个群体的哪种健康问题，并请列出活动发布的指导性文件名称。 

“启明行动”是为了防控儿童青少年的近视问题，并发布了《防控儿童青少年近视核心知识十条》。 



# 构建RAG

In [4]:
def pretty_print_docs(docs):
  print(
      f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n + {d.page_content}\n{d.metadata}" for i,d in enumerate(docs)])
  )

In [5]:
from langchain.schema import Document

# 加载文档
docs = [Document(page_content=origin_data['news1'], metadata={'source': origin_data['ID']})]
print(len(docs))

1


In [6]:
pretty_print_docs(docs[:2])

Document 1:

 + 2023-07-28 10:14:27作者：白剑峰来源：人民日报    ，正文：为在全社会形成重视儿童眼健康的良好氛围，持续推进综合防控儿童青少年近视工作落实，国家卫生健康委决定在全国持续开展“启明行动”——防控儿童青少年近视健康促进活动，并发布了《防控儿童青少年近视核心知识十条》。本次活动的主题为：重视儿童眼保健，守护孩子明眸“视”界。强调预防为主，推动关口前移，倡导和推动家庭及全社会共同行动起来，营造爱眼护眼的视觉友好环境，共同呵护好孩子的眼睛，让他们拥有一个光明的未来。国家卫生健康委要求，开展社会宣传和健康教育。充分利用网络、广播电视、报纸杂志、海报墙报、培训讲座等多种形式，广泛开展宣传倡导，向社会公众传播开展儿童眼保健、保护儿童视力健康的重要意义，以《防控儿童青少年近视核心知识十条》为重点普及预防近视科学知识。创新健康教育方式和载体，开发制作群众喜闻乐见的健康教育科普作品，利用互联网媒体扩大传播效果，提高健康教育的针对性、精准性和实效性。指导相关医疗机构将儿童眼保健和近视防控等科学知识纳入孕妇学校、家长课堂内容。开展儿童眼保健及视力检查咨询指导。医疗机构要以儿童家长和养育人为重点，结合眼保健和眼科临床服务，开展个性化咨询指导。要针对儿童常见眼病和近视防控等重点问题，通过面对面咨询指导，引导儿童家长树立近视防控意识，改变不良生活方式，加强户外活动，养成爱眼护眼健康行为习惯。提高儿童眼保健专科服务能力。各地要积极推进儿童眼保健专科建设，扎实组织好妇幼健康职业技能竞赛“儿童眼保健”项目，推动各层级开展比武练兵，提升业务能力。
{'source': '64fa9b27b82641eb8ecbe14c'}


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 分割文档
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=50,
)
split_docs = text_splitter.split_documents(docs)
print(len(split_docs))

4


In [8]:
pretty_print_docs(split_docs[:2])

Document 1:

 + 2023-07-28 10:14:27作者：白剑峰来源：人民日报
{'source': '64fa9b27b82641eb8ecbe14c'}
----------------------------------------------------------------------------------------------------
Document 2:

 + ，正文：为在全社会形成重视儿童眼健康的良好氛围，持续推进综合防控儿童青少年近视工作落实，国家卫生健康委决定在全国持续开展“启明行动”——防控儿童青少年近视健康促进活动，并发布了《防控儿童青少年近视核心知识十条》。本次活动的主题为：重视儿童眼保健，守护孩子明眸“视”界。强调预防为主，推动关口前移，倡导和推动家庭及全社会共同行动起来，营造爱眼护眼的视觉友好环境，共同呵护好孩子的眼睛，让他们拥有一个光明的未来。国家卫生健康委要求，开展社会宣传和健康教育。充分利用网络、广播电视、报纸杂志、海报墙报、培训讲座等多种
{'source': '64fa9b27b82641eb8ecbe14c'}


In [9]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

# 创建嵌入模型
model_name = 'BAAI/bge-large-zh-v1.5'
model_kwargs = {'device': 'cuda'}  # 需要安装GPU版本的torch 
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from langchain_community.vectorstores import FAISS

# 创建向量存储
vectordb = FAISS.from_documents(split_docs, embeddings)

In [11]:
index_folder_path = "data/faiss_index"
index_name = "0"

In [12]:
# 保存索引
vectordb.save_local(index_folder_path, index_name)

In [13]:
# 加载索引
vectordb = FAISS.load_local(index_folder_path, embeddings, index_name)

In [14]:
# 创建检索器
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [15]:
from langchain_openai import ChatOpenAI

# 创建模型
llm = ChatOpenAI(temperature=0)
print(llm.model_name)

gpt-3.5-turbo


In [16]:
from langchain.chains import RetrievalQA

# 创建链
chain = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True)

In [17]:
# 运行链
response = chain(origin_data['questions'])
print(response)

{'query': '国家卫生健康委在2023年7月28日开展的“启明行动”是为了防控哪个群体的哪种健康问题，并请列出活动发布的指导性文件名称。', 'result': '国家卫生健康委在2023年7月28日开展的“启明行动”是为了防控儿童青少年近视健康问题。活动发布的指导性文件名称是《防控儿童青少年近视核心知识十条》。', 'source_documents': [Document(page_content='，正文：为在全社会形成重视儿童眼健康的良好氛围，持续推进综合防控儿童青少年近视工作落实，国家卫生健康委决定在全国持续开展“启明行动”——防控儿童青少年近视健康促进活动，并发布了《防控儿童青少年近视核心知识十条》。本次活动的主题为：重视儿童眼保健，守护孩子明眸“视”界。强调预防为主，推动关口前移，倡导和推动家庭及全社会共同行动起来，营造爱眼护眼的视觉友好环境，共同呵护好孩子的眼睛，让他们拥有一个光明的未来。国家卫生健康委要求，开展社会宣传和健康教育。充分利用网络、广播电视、报纸杂志、海报墙报、培训讲座等多种', metadata={'source': '64fa9b27b82641eb8ecbe14c'}), Document(page_content='家卫生健康委要求，开展社会宣传和健康教育。充分利用网络、广播电视、报纸杂志、海报墙报、培训讲座等多种形式，广泛开展宣传倡导，向社会公众传播开展儿童眼保健、保护儿童视力健康的重要意义，以《防控儿童青少年近视核心知识十条》为重点普及预防近视科学知识。创新健康教育方式和载体，开发制作群众喜闻乐见的健康教育科普作品，利用互联网媒体扩大传播效果，提高健康教育的针对性、精准性和实效性。指导相关医疗机构将儿童眼保健和近视防控等科学知识纳入孕妇学校、家长课堂内容。开展儿童眼保健及视力检查咨询指导。医疗机构要以儿童家长和养育', metadata={'source': '64fa9b27b82641eb8ecbe14c'}), Document(page_content='控等科学知识纳入孕妇学校、家长课堂内容。开展儿童眼保健及视力检查咨询指导。医疗机构要以儿童家长和养育人为重点，结合眼保健和眼科临床服务，开展个性化咨询指导。要针对儿童常见眼病和近视防控等重点问题，通过面对面咨询指导，引导儿童家长树立近视防控意识，改变不良生活

In [18]:
response['query']

'国家卫生健康委在2023年7月28日开展的“启明行动”是为了防控哪个群体的哪种健康问题，并请列出活动发布的指导性文件名称。'

In [19]:
response['result']

'国家卫生健康委在2023年7月28日开展的“启明行动”是为了防控儿童青少年近视健康问题。活动发布的指导性文件名称是《防控儿童青少年近视核心知识十条》。'

In [20]:
pretty_print_docs(response['source_documents'])

Document 1:

 + ，正文：为在全社会形成重视儿童眼健康的良好氛围，持续推进综合防控儿童青少年近视工作落实，国家卫生健康委决定在全国持续开展“启明行动”——防控儿童青少年近视健康促进活动，并发布了《防控儿童青少年近视核心知识十条》。本次活动的主题为：重视儿童眼保健，守护孩子明眸“视”界。强调预防为主，推动关口前移，倡导和推动家庭及全社会共同行动起来，营造爱眼护眼的视觉友好环境，共同呵护好孩子的眼睛，让他们拥有一个光明的未来。国家卫生健康委要求，开展社会宣传和健康教育。充分利用网络、广播电视、报纸杂志、海报墙报、培训讲座等多种
{'source': '64fa9b27b82641eb8ecbe14c'}
----------------------------------------------------------------------------------------------------
Document 2:

 + 家卫生健康委要求，开展社会宣传和健康教育。充分利用网络、广播电视、报纸杂志、海报墙报、培训讲座等多种形式，广泛开展宣传倡导，向社会公众传播开展儿童眼保健、保护儿童视力健康的重要意义，以《防控儿童青少年近视核心知识十条》为重点普及预防近视科学知识。创新健康教育方式和载体，开发制作群众喜闻乐见的健康教育科普作品，利用互联网媒体扩大传播效果，提高健康教育的针对性、精准性和实效性。指导相关医疗机构将儿童眼保健和近视防控等科学知识纳入孕妇学校、家长课堂内容。开展儿童眼保健及视力检查咨询指导。医疗机构要以儿童家长和养育
{'source': '64fa9b27b82641eb8ecbe14c'}
----------------------------------------------------------------------------------------------------
Document 3:

 + 控等科学知识纳入孕妇学校、家长课堂内容。开展儿童眼保健及视力检查咨询指导。医疗机构要以儿童家长和养育人为重点，结合眼保健和眼科临床服务，开展个性化咨询指导。要针对儿童常见眼病和近视防控等重点问题，通过面对面咨询指导，引导儿童家长树立近视防控意识，改变不良生活方式，加强户外活动，养成爱眼护眼健康行为习惯。提高儿童眼保健

# 评估RAG

[使用您的测试集进行评估](https://docs.ragas.io/en/stable/getstarted/evaluation.html)

数据集包含以下列：

- question: list[str] - 这些是将评估您的 RAG 管道的问题。
- answer: list[str] - 您的 RAG 管道生成的答案。
- context: list[list[str]] - 传递到 LLM 来回答问题的上下文。
- ground_truth: list[str] - 问题的真实答案。


In [22]:
# 构建评估数据集
from datasets import Dataset

questions = [origin_data['questions']]
answers = [response['result']]
contexts = [[doc.page_content for doc in response['source_documents']]]
ground_truths = [origin_data['answers']]

evaluate_data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}

evaluate_dataset = Dataset.from_dict(evaluate_data)
evaluate_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 1
})

In [23]:
evaluate_dataset[0]

{'question': '国家卫生健康委在2023年7月28日开展的“启明行动”是为了防控哪个群体的哪种健康问题，并请列出活动发布的指导性文件名称。',
 'answer': '国家卫生健康委在2023年7月28日开展的“启明行动”是为了防控儿童青少年近视健康问题。活动发布的指导性文件名称是《防控儿童青少年近视核心知识十条》。',
 'contexts': ['，正文：为在全社会形成重视儿童眼健康的良好氛围，持续推进综合防控儿童青少年近视工作落实，国家卫生健康委决定在全国持续开展“启明行动”——防控儿童青少年近视健康促进活动，并发布了《防控儿童青少年近视核心知识十条》。本次活动的主题为：重视儿童眼保健，守护孩子明眸“视”界。强调预防为主，推动关口前移，倡导和推动家庭及全社会共同行动起来，营造爱眼护眼的视觉友好环境，共同呵护好孩子的眼睛，让他们拥有一个光明的未来。国家卫生健康委要求，开展社会宣传和健康教育。充分利用网络、广播电视、报纸杂志、海报墙报、培训讲座等多种',
  '家卫生健康委要求，开展社会宣传和健康教育。充分利用网络、广播电视、报纸杂志、海报墙报、培训讲座等多种形式，广泛开展宣传倡导，向社会公众传播开展儿童眼保健、保护儿童视力健康的重要意义，以《防控儿童青少年近视核心知识十条》为重点普及预防近视科学知识。创新健康教育方式和载体，开发制作群众喜闻乐见的健康教育科普作品，利用互联网媒体扩大传播效果，提高健康教育的针对性、精准性和实效性。指导相关医疗机构将儿童眼保健和近视防控等科学知识纳入孕妇学校、家长课堂内容。开展儿童眼保健及视力检查咨询指导。医疗机构要以儿童家长和养育',
  '控等科学知识纳入孕妇学校、家长课堂内容。开展儿童眼保健及视力检查咨询指导。医疗机构要以儿童家长和养育人为重点，结合眼保健和眼科临床服务，开展个性化咨询指导。要针对儿童常见眼病和近视防控等重点问题，通过面对面咨询指导，引导儿童家长树立近视防控意识，改变不良生活方式，加强户外活动，养成爱眼护眼健康行为习惯。提高儿童眼保健专科服务能力。各地要积极推进儿童眼保健专科建设，扎实组织好妇幼健康职业技能竞赛“儿童眼保健”项目，推动各层级开展比武练兵，提升业务能力。',
  '2023-07-28 10:14:27作者：白剑峰来源：人民日报'],
 'ground_truth': '“启明行动”是


[评估指标](https://docs.ragas.io/en/stable/concepts/metrics/index.html)
- Faithfulness
  - 如果答案中提出的所有主张都可以从给定的上下文中推断出来，则生成的答案被认为是忠实的。
  - answer 和 contexts
- Answer Relevancy
  - 评估指标“答案相关性”重点评估生成的答案与给定提示的相关程度。不完整或包含冗余信息的答案将获得较低分数。
  - answer 和 question
- Context Precision
  - 用于评估 contexts 中存在的所有真实相关项目是否排名较高。理想情况下，所有相关块必须出现在顶层。
  - question 和 contexts
- Context Recall
  - 上下文回忆衡量检索到的上下文与带注释的答案（被视为基本事实）的一致程度。
  - ground truth 和 contexts
---

- RAGAS评估框架评估了RAG管道的两个主要组件：
    - Retriever 检索器
    - Generator 生成器

---

- 与评估 Retrieval 相关的指标如下：
    - Context Precision。question 和 contexts
    - Context Recall。ground truth 和 contexts
- 与评估 Generation 相关的指标如下：
  - Faithfulness。answer 和 contexts
  - Answer Relevancy。answer 和 question

In [24]:
# 导入指标
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)


# 开始评估
from ragas import evaluate

evaluate_result = evaluate(
    evaluate_dataset, 
    metrics=[
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ]
)
evaluate_result

Evaluating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:19<00:00,  4.93s/it]


{'faithfulness': 1.0000, 'answer_relevancy': 0.9222, 'context_recall': 1.0000, 'context_precision': 1.0000}

In [25]:
df_evaluate_result = evaluate_result.to_pandas()
df_evaluate_result

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision
0,国家卫生健康委在2023年7月28日开展的“启明行动”是为了防控哪个群体的哪种健康问题，并请...,国家卫生健康委在2023年7月28日开展的“启明行动”是为了防控儿童青少年近视健康问题。活动...,[，正文：为在全社会形成重视儿童眼健康的良好氛围，持续推进综合防控儿童青少年近视工作落实，国...,“启明行动”是为了防控儿童青少年的近视问题，并发布了《防控儿童青少年近视核心知识十条》。,1.0,0.922201,1.0,1.0


In [26]:
df_evaluate_result.to_excel(f'data/{index_name}_eval_result.xlsx', index=False)